## A Simulated IOT Federated Learning Framework for Forest Fire Prediction

Forest fire ignition prediction is essential to the safety of communities and the proper allocation of fire fighting resources. This notebook simulates a network of clients that collect local meteorological data. Each client participates in a federated learning system where a client trains a machine learning model to classify whether the area will have a forest fire ignition. The model weights of each client are shared with a central server that averages the weights and shares them back to the clients.

This notebook demonstrates the two machine learning types shown below.

|<center>name|<center>server|<center>clients|<center>training|<center>training data|<center>evaluation type|<center>evaluation|<center>evaluation data|
|---|---|---|---|---|---|---|---|
|<center>central server ML|<center>yes|<center>no|<center>on server|<center>on server|<center>centralized|<center>on server|<center>on server|
|<center>federated ML (Federated Eval)|<center>yes|<center>yes|<center>on clients|<center>on clients|<center>federated|<center>on clients|<center>on clients|


The data used is from the paper: *Framework for Creating Forest Fire Ignition Prediction Datasets.* Each row represents meteorological data at a geographical location at a specific time. TODO: Add table example.

Much of the code used in this notebook is based on the Flower code examples located [here](https://github.com/adap/flower/tree/main/examples) and the Keras timeseries tutorials located [here](https://keras.io/examples/timeseries/).
The code below is very much a work in progress. 

In [51]:
#if this file is being used in colab set to 1 otherwise 0
using_colab = 1

In [52]:
if (using_colab == 1):
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
if (using_colab ==1):
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [54]:
#load libraries
import math
import os
import glob
import gc
import datetime
import typing
from typing import List
from typing import Tuple
from typing import Dict
from typing import Optional
import random
import tempfile
from tables.file import File

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Model

In [55]:
if (using_colab == 1):
    !pip install -q flwr[simulation] pandas

In [56]:
import flwr as fl
from flwr.common import Metrics

In [57]:
#overall environment settings

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Training on GPU or CPU?
#tf.config.set_visible_devices([], 'CPU')
print(
    f"Training on {'GPU' if tf.config.get_visible_devices('GPU') else 'CPU'} using TensorFlow {tf.__version__} and Flower {fl.__version__}"
)

Training on CPU using TensorFlow 2.12.0 and Flower 1.4.0


In [58]:
#global variables

ml_type = 0 # classic ML = 0, federated ML w/ centralized evaluation = 1, federated ML w/ federated eval = 2

federated_path = "<path to your client datasets>" 
centralized_path = "<path to your server dataset"
results_path = "<path to where you want to store results>"


cid = str(0) # preliminary client id
log_dir = "<path to where you want to store Tensorflow logs>" + cid + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 

downsample_test_set = 0 # 0 if test set not downsampled, 1 otherwise

# TODO: Add the below to a client config file
NUM_CLIENTS = 24
num_rounds = 3
epochs = 10 # the number of epochs classic machine learning will use
fl_epochs = 3

sequence_len = 120 # 5 days * 24 hours
past_len = sequence_len
future_len = 24 # 1*24 hours
sampling_rate = 1 # in time series conversion use every row (hour) in loaded datasets
sequence_stride = 1 # in time series conversion each series is this far apart

#fraction_fit=0.75,  # Sample 10% of available clients for training
#fraction_evaluate=0.25,  # Sample 5% of available clients for evaluation
#min_fit_clients=12,  # Never sample less than 10 clients for training
#min_evaluate_clients=6,  # Never sample less than 5 clients for evaluation
#min_available_clients=12,  # Wait until at least 75 clients are available

#create an array for results
np.set_printoptions(suppress=True)
results = np.empty([13,2])

## Data loading functions

In [59]:
def normalize_data(x):
    """Normalizes the data of an array by column. 
    Shifts and scales inputs into a distribution centered around 0 
    with standard deviation 1.

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        The original array, but normalized.
    """
    data = x
    layer = layers.Normalization()
    layer.adapt(data)
    features_normalized = layer(data)
    return features_normalized

In [60]:
def mask_create(x):
    """Finds the class count of the input array and creates a mask that can be used
    to randomly downsample an array of labels so that the number of 
    negative labels = the number of positive labels. 

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        A masked version of the input array.
    """
    mask_length = x.shape[0]
    mask = tf.reshape(x, [mask_length])
    y, idx, class_count = tf.unique_with_counts(mask)
    ignition_count = tf.get_static_value(class_count[1])
    mask = mask.numpy()
    count = 0 
    while count < ignition_count:
        #rand_num = random.randint(0,mask_length)
        rand_num = random.randint(1, mask_length-1)
        if (mask[rand_num] == 0):
            mask[rand_num] = 1
            count += 1
    return mask

In [61]:
def load_datasets(path: str):
    """Loads all the csv datasets in a folder.
    The loaded data is divided into train, validation, and test sets.
    The data is turned into time series data
    All the data is normalized.
    Train and validation datasets are downsampled.
    TODO: divide this function into smaller functions

    Parameters
    ----------
    path: string 
        The path to the dataset folder.
        
    Returns
    -------
    train_x, train_y, val_x, val_y, test_x, test_y : NDarrays
        A masked version of the input array.
    """

    train_x = []
    train_y = []
    val_x = []
    val_y = []
    test_x = []
    test_y = []

    #load data
    for filename in glob.glob(os.path.join(path, '*.csv')):
        print("\nnow reading " + filename + "\n")
        #read file
        df = pd.read_csv(filename, index_col=[0])
        
        df_train = df[(df['year'] < 2001)]
        df_val = df[(df['year'] > 2001) & (df['year'] < 2012)]
        df_test = df[(df['year'] >= 2012)]
        
        features = ['stl2', 't2m', 'stl1', 'stl3', 'skt', 'swvl1', 'd2m', 'swvl2']
        train_features = df_train[features]
        train_labels = df_train[["ignition"]]
        val_features = df_val[features]
        val_labels = df_val[["ignition"]]
        test_features = df_test[features]
        test_labels = df_test[["ignition"]]
        #convert to numpy
        train_features = train_features.values
        val_features = val_features.values
        test_features = test_features.values

        #normalize
        train_features_normalize = normalize_data(train_features)
        val_features_normalize = normalize_data(val_features)
        test_features_normalize = normalize_data(test_features)
        
        #we want to predict at a future point
        #so we clip the length of the features plus the hours till the future point
        start = past_len + future_len
        train_labels = train_labels.iloc[start:].values
        val_labels = val_labels.iloc[start:].values
        test_labels = test_labels.iloc[start:].values
        
        batch_size = 107856 #factor of 5136 (321 * 16)
        #convert to time series data
        train_dataset = keras.utils.timeseries_dataset_from_array(
            train_features_normalize,
            train_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        val_dataset = keras.utils.timeseries_dataset_from_array(
            val_features_normalize,
            val_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        test_dataset = keras.utils.timeseries_dataset_from_array(
            test_features_normalize,
            test_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)
        
        #for bookkeeping print out the shapes of the datasets
        for train_features, train_labels in train_dataset:
            print("train_dataset features shape:", train_features.shape)
            print("targets_dataset labels shape:", train_labels.shape)
            break

        for val_features, val_labels in val_dataset:
            print("\nval_dataset features shape:", val_features.shape)
            print("val_dataset labels shape:", val_labels.shape)
            break

        for test_features, test_labels in test_dataset:
            print("\ntest_dataset features shape:", test_features.shape)
            print("test_dataset labels shape:", test_labels.shape)
            break
       
        # randomly downsample the data using masks
        train_mask = mask_create(train_labels)
        train_features_masked = tf.boolean_mask(train_features, train_mask)
        train_labels_masked = tf.boolean_mask(train_labels, train_mask)
        
        val_mask = mask_create(val_labels)
        val_features_masked = tf.boolean_mask(val_features, val_mask)
        val_labels_masked = tf.boolean_mask(val_labels, val_mask)
        
        test_mask = mask_create(test_labels)
        test_features_masked = tf.boolean_mask(test_features, test_mask)
        test_labels_masked = tf.boolean_mask(test_labels, test_mask)
        
        train_x.append(train_features_masked)
        train_y.append(train_labels_masked)
        val_x.append(val_features_masked)
        val_y.append(val_labels_masked)
        if (downsample_test_set == 1):
            test_x.append(test_features_masked)
            test_y.append(test_labels_masked)
        else:
            test_x.append(test_features)
            test_y.append(test_labels)
        
    print("\nDone loading data.\n")
    return train_x, train_y, val_x, val_y, test_x, test_y 



In [62]:
def get_value_count(x):
    """A helper function that returns the count of class labels.

    Parameters
    ----------
    x: NDArray 
        An array with class labels.
        
    Returns
    -------
    non_ignition_count, ignition_count : int
        The counts of the ignition class.
    """
    length = x[0].shape[0]
    x = tf.reshape(x, [length])
    y, idx, class_count = tf.unique_with_counts(x)
    non_ignition_count = tf.get_static_value(class_count[0])
    ignition_count = tf.get_static_value(class_count[1])
    return non_ignition_count, ignition_count
    

## Model and Metrics

In [63]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
]

def make_model(metrics=METRICS):
    inputs = keras.Input(shape=(sequence_len, trainloaders_x[0].shape[2]))
    x = layers.LSTM(8, activation='sigmoid')(inputs)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)
        
    return model


## Federated Learning Functions

In [64]:
class FlowerClient(fl.client.NumPyClient):
    """Instantiates a FlowerClient object

    Parameters
    ----------
    fl.client: NumPyClient
        A premade helper client.
    """
    def __init__(self, cid, model, x_train, y_train, x_val, y_val, x_test, y_test, tb_callback) -> None:
        self.cid = cid
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
        self.x_test, self.y_test = x_test, y_test
        self.tb_callback = tb_callback

    def get_parameters(self, config):
        return self.model.get_weights()

    # presetting config allows us to use FlowerClient with classic ml.
    def fit(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        print("in fit")
        server_round = config["server_round"]
        print("in server round: ", server_round)
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=epochs, verbose=2, callbacks=self.tb_callback, validation_data=(self.x_val, self.y_val),)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        self.model.set_weights(parameters)
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc  = self.model.evaluate(self.x_test, self.y_test, verbose=2)
        print("Hi, the loss is ", loss)
        return loss, len(self.x_val), {"tp": tp,
                                       "fp": fp,
                                       "tn": tn,
                                       "fn": fn,
                                       "accuracy": accuracy,
                                       "precision": precision,
                                       "recall": recall,
                                       "auc": auc,
                                       "prc": prc
                                      }

    # this method allows metrics to be passed when using a single server with no clients
    def evaluate_single(self, x_test, y_test):
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = self.model.evaluate(x_test, y_test, verbose=2)
        return loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc

In [65]:
def client_fn(cid: str) -> fl.client.Client:
    """Instantiates the client object. 

    Parameters
    ----------
    cid: str 
        The client id.
        
    Returns
    -------
    cid : NDarray
    model: model
    test/val/test datasets: NDarray
    tensorboard: tensorflow callbacks
        Everything a client needs to create a ML model
    """

    print("\nThis is client: ", cid)

    x_train_cid = trainloaders_x[int(cid)]
    y_train_cid = trainloaders_y[int(cid)]
    x_val_cid = valloaders_x[int(cid)]
    y_val_cid = valloaders_y[int(cid)]
    x_test_cid = testloaders_x[int(cid)]
    y_test_cid = testloaders_y[int(cid)]

    print("Loaded data for client: ", cid, "\n")

    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
    ]

    def make_model(metrics=METRICS):
        inputs = keras.Input(shape=(sequence_len, x_train_cid.shape[2]))
        x = layers.LSTM(8, activation='sigmoid')(inputs)
        x = layers.Flatten()(x)
        outputs = layers.Dense(1, activation="sigmoid")(x)
        model = keras.Model(inputs, outputs)

        model.compile(
            optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics)
        
        return model
    
    print("Making model: ", cid)
    model = make_model()
    
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    tb_callback = keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)

    # Create and return client
    print("\nClient CID: " + str(cid) + " is done.\n")
    return FlowerClient(cid, model, x_train_cid, y_train_cid, x_val_cid, y_val_cid, x_test_cid, y_test_cid, tensorboard)

In [68]:
# TODO: this function isn't working correctly, have a look at it
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    """A function for the final evaluation of the simulation. 
    All metrics are averaged and output.

    Parameters
    ----------
    metrics: List 
        A list of the metrics being used
        
    Returns
    -------
     : Dict
        A dictionary of the averaged results from each round.
    """
    # Multiply each metric of each client by number of examples used
    tps = [num_examples * m["tp"] for num_examples, m in metrics]
    fps = [num_examples * m["fp"] for num_examples, m in metrics]
    tns = [num_examples * m["tn"] for num_examples, m in metrics]
    fns = [num_examples * m["fn"] for num_examples, m in metrics]
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    precisions = [num_examples * m["precision"] for num_examples, m in metrics]
    recalls = [num_examples * m["recall"] for num_examples, m in metrics]
    aucs = [num_examples * m["auc"] for num_examples, m in metrics]
    prcs = [num_examples * m["prc"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"tp": sum(tps) / sum(examples),
            "fp": sum(fps) / sum(examples),
            "tn": sum(tns) / sum(examples),
            "fn": sum(fns) / sum(examples),
            "accuracy": sum(accuracies) / sum(examples),
            "precision": sum(precisions) / sum(examples),
            "recall": sum(recalls) / sum(examples),
            "auc": sum(aucs) / sum(examples),
            "prc": sum(prcs) / sum(examples)
           }

In [69]:
def fit_config(server_round: int):
    """Passes information from the server to client.

    Parameters
    ----------
    server_round: int
        What server round simulation is in.
        
    Returns
    -------
    config : Dict
        A dictionary of values.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        #"local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

## Launch classic machine learning

In [70]:
# load the dataset for centralized evaluation (for classic ml training and testing) 
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(centralized_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/01_clients/dly_avg_1of1_50.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

Done loading data.



In [71]:
count = get_value_count(trainloaders_y)
print("Train set nonignitions and ignitions are:", count)
count = get_value_count(valloaders_y)
print("Validation set nonignitions and ignitions are:", count)
count = get_value_count(testloaders_y)
print("Test set nonignitions and ignitions are:", count)

Train set nonignitions and ignitions are: (2811, 2811)
Validation set nonignitions and ignitions are: (2534, 2534)
Test set nonignitions and ignitions are: (44719, 1361)


In [72]:
# create an initial model to prime flower client
model = make_model()

In [73]:
# train single server classic ml
# create a flower client that represents a classic ml single server
central_server_model = client_fn(str(0))
# set parameters for classic_ml so it can use single flower client function
parameters = model.get_weights()
# run fit
config = fit_config(0)
history = central_server_model.fit(parameters, config)


This is client:  0
Loaded data for client:  0 

Making model:  0

Client CID: 0 is done.

in fit
in server round:  0
Epoch 1/10
176/176 - 10s - loss: 0.6397 - tp: 1989.0000 - fp: 1135.0000 - tn: 1676.0000 - fn: 822.0000 - accuracy: 0.6519 - precision: 0.6367 - recall: 0.7076 - auc: 0.7053 - prc: 0.6994 - val_loss: 0.6202 - val_tp: 1838.0000 - val_fp: 931.0000 - val_tn: 1603.0000 - val_fn: 696.0000 - val_accuracy: 0.6790 - val_precision: 0.6638 - val_recall: 0.7253 - val_auc: 0.7163 - val_prc: 0.6838 - 10s/epoch - 60ms/step
Epoch 2/10
176/176 - 7s - loss: 0.6210 - tp: 1895.0000 - fp: 968.0000 - tn: 1843.0000 - fn: 916.0000 - accuracy: 0.6649 - precision: 0.6619 - recall: 0.6741 - auc: 0.7147 - prc: 0.7116 - val_loss: 0.6138 - val_tp: 1773.0000 - val_fp: 850.0000 - val_tn: 1684.0000 - val_fn: 761.0000 - val_accuracy: 0.6821 - val_precision: 0.6759 - val_recall: 0.6997 - val_auc: 0.7232 - val_prc: 0.6936 - 7s/epoch - 38ms/step
Epoch 3/10
176/176 - 7s - loss: 0.6167 - tp: 1870.0000 - fp: 

In [74]:
#evaluate single server classic ml
loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = central_server_model.evaluate_single(testloaders_x[0], testloaders_y[0])

1440/1440 - 14s - loss: 0.5937 - tp: 930.0000 - fp: 12505.0000 - tn: 32214.0000 - fn: 431.0000 - accuracy: 0.7193 - precision: 0.0692 - recall: 0.6833 - auc: 0.7574 - prc: 0.0844 - 14s/epoch - 10ms/step


In [75]:
print("Single server classic ml evaluation\n\
\ttp:\t%d\n\
\tfp:\t%d\n\
\ttn:\t%d\n\
\tfn:\t%d\n\n\
\tloss:\t%f\n\
\tacc:\t%f\n\
\tprec:\t%f\n\
\trec:\t%f\n\
\tauc:\t%f\n\
\tprc:\t%f\
" % (tp,fp,tn,fn,loss,accuracy,precision,recall,auc,prc))

Single server classic ml evaluation
	tp:	930
	fp:	12505
	tn:	32214
	fn:	431

	loss:	0.593685
	acc:	0.719271
	prec:	0.069222
	rec:	0.683321
	auc:	0.757354
	prc:	0.084407


In [76]:
#write to results array
results[0] = tp
results[1][0] = fp
results[2][0] = tn
results[3][0] = fn
results[4][0] = accuracy
results[5][0] = precision
results[6][0] = recall
results[7][0] = auc
results[8][0] = prc
results[9][0] = epochs
results[10][0] = 1
results[11][0] = 1
results[12][0] = 1

In [30]:
#remove data from memory that isn't needed
for_removal = [trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y]
del(for_removal)
del(model)
del(central_server_model)
gc.collect()
print('Central server data removed.')

2260

## Federated machine learning

The Flower federated learning framework is used from here down. More info can be found at https://flower.dev

In [31]:
#TODO: there is a random out-of-bounds error with masking, for now if it occurs run this cell again
#load the dataset for federated learning
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(federated_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/121.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/120.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/119.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (460

In [38]:
#change the number of epochs from single server to federated
epochs = fl_epochs

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.75,  # Sample 10% of available clients for training
    fraction_evaluate=0.25,  # Sample 5% of available clients for evaluation
    min_fit_clients=12,  # Never sample less than 10 clients for training
    min_evaluate_clients=6,  # Never sample less than 5 clients for evaluation
    min_available_clients=12,  # Wait until at least 75 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    on_fit_config_fn=fit_config, # Pass the fit_config function
)

# Start simulation
fl_history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
)


INFO flwr 2023-06-09 17:35:04,838 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-06-09 17:35:11,186	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-09 17:35:12,337 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 16267962777.0, 'memory': 32535925556.0, 'node:172.28.0.12': 1.0, 'CPU': 8.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 16267962777.0, 'memory': 32535925556.0, 'node:172.28.0.12': 1.0, 'CPU': 8.0}
INFO flwr 2023-06-09 17:35:12,341 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-09 17:35:12,343 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
INFO flwr 2023-06-09 17:35:16,809 | server.py:277 | Receiv

(launch_and_get_parameters pid=42754) 
(launch_and_get_parameters pid=42754) This is client:  19
(launch_and_get_parameters pid=42754) Loaded data for client:  19 
(launch_and_get_parameters pid=42754) 
(launch_and_get_parameters pid=42754) Making model:  19
(launch_and_get_parameters pid=42754) 
(launch_and_get_parameters pid=42754) Client CID: 19 is done.
(launch_and_get_parameters pid=42754) 
(launch_and_fit pid=42754) 
(launch_and_fit pid=42754) This is client:  23
(launch_and_fit pid=42754) Loaded data for client:  23 
(launch_and_fit pid=42754) 
(launch_and_fit pid=42754) Making model:  23
(launch_and_fit pid=42754) 
(launch_and_fit pid=42754) Client CID: 23 is done.
(launch_and_fit pid=42754) 
(launch_and_fit pid=42754) in fit
(launch_and_fit pid=42754) in server round:  1
(launch_and_fit pid=42754) Epoch 1/3
(launch_and_fit pid=42751)  [repeated 2x across cluster]
(launch_and_fit pid=42751) Making model:  15
(launch_and_fit pid=42751) Making model:  15
(launch_and_fit pid=42751

(raylet) Spilled 4322 MiB, 1 objects, write throughput 62 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


(launch_and_fit pid=42754) 
(launch_and_fit pid=42754) This is client:  20
(launch_and_fit pid=42754) Loaded data for client:  20 
(launch_and_fit pid=42754) 
(launch_and_fit pid=42754) Making model:  20
(launch_and_fit pid=42754) 
(launch_and_fit pid=42754) Client CID: 20 is done.
(launch_and_fit pid=42754) 
(launch_and_fit pid=42754) in fit
(launch_and_fit pid=42754) in server round:  1
(launch_and_fit pid=42754) Epoch 1/3


(raylet) Spilled 8644 MiB, 2 objects, write throughput 119 MiB/s.


...

(launch_and_fit pid=42751) This is client:  7
(launch_and_fit pid=42751) Loaded data for client:  7 
(launch_and_fit pid=42751) Making model:  7
(launch_and_fit pid=42751) 51/51 - 2s - loss: 0.5879 - tp: 587.0000 - fp: 258.0000 - tn: 543.0000 - fn: 214.0000 - accuracy: 0.7054 - precision: 0.6947 - recall: 0.7328 - auc: 0.7606 - prc: 0.7593 - val_loss: 0.5602 - val_tp: 495.0000 - val_fp: 202.0000 - val_tn: 420.0000 - val_fn: 127.0000 - val_accuracy: 0.7355 - val_precision: 0.7102 - val_recall: 0.7958 - val_auc: 0.7906 - val_prc: 0.7271 - 2s/epoch - 48ms/step [repeated 2x across cluster]
(launch_and_fit pid=42751)  [repeated 2x across cluster]
(launch_and_fit pid=42751) Client CID: 7 is done.
(launch_and_fit pid=42751) in fit
(launch_and_fit pid=42751) in server round:  3
(launch_and_fit pid=42751) Epoch 1/3
(launch_and_fit pid=42754) 36/36 - 7s - loss: 0.6149 - tp: 372.0000 - fp: 183.0000 - tn: 382.0000 - fn: 193.0000 - accuracy: 0.6673 - precision: 0.6703 - recall: 0.6584 - auc: 0.7224

DEBUG flwr 2023-06-09 17:55:33,265 | server.py:232 | fit_round 3 received 18 results and 0 failures
DEBUG:flwr:fit_round 3 received 18 results and 0 failures
DEBUG flwr 2023-06-09 17:55:33,290 | server.py:168 | evaluate_round 3: strategy sampled 6 clients (out of 24)
DEBUG:flwr:evaluate_round 3: strategy sampled 6 clients (out of 24)


(launch_and_fit pid=42751) 43/43 - 2s - loss: 0.5517 - tp: 509.0000 - fp: 203.0000 - tn: 480.0000 - fn: 174.0000 - accuracy: 0.7240 - precision: 0.7149 - recall: 0.7452 - auc: 0.7978 - prc: 0.8000 - val_loss: 0.5895 - val_tp: 254.0000 - val_fp: 104.0000 - val_tn: 253.0000 - val_fn: 103.0000 - val_accuracy: 0.7101 - val_precision: 0.7095 - val_recall: 0.7115 - val_auc: 0.7543 - val_prc: 0.7224 - 2s/epoch - 40ms/step
(launch_and_evaluate pid=42754) 
(launch_and_evaluate pid=42754) This is client:  19
(launch_and_evaluate pid=42754) Loaded data for client:  19 
(launch_and_evaluate pid=42754) 
(launch_and_evaluate pid=42754) Making model:  19
(launch_and_evaluate pid=42754) Client CID: 19 is done.
(launch_and_evaluate pid=42754) 1440/1440 - 18s - loss: 0.6027 - tp: 238.0000 - fp: 14905.0000 - tn: 30864.0000 - fn: 73.0000 - accuracy: 0.6750 - precision: 0.0157 - recall: 0.7653 - auc: 0.7587 - prc: 0.0162 - 18s/epoch - 13ms/step
(launch_and_evaluate pid=42754) Hi, the loss is  0.60273200273

DEBUG flwr 2023-06-09 17:57:44,579 | server.py:182 | evaluate_round 3 received 6 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 6 results and 0 failures
INFO flwr 2023-06-09 17:57:44,582 | server.py:147 | FL finished in 1347.763735433
INFO:flwr:FL finished in 1347.763735433
INFO flwr 2023-06-09 17:57:44,585 | app.py:218 | app_fit: losses_distributed [(1, 0.6456963047236771), (2, 0.6226884936722455), (3, 0.6001743322193792)]
INFO:flwr:app_fit: losses_distributed [(1, 0.6456963047236771), (2, 0.6226884936722455), (3, 0.6001743322193792)]
INFO flwr 2023-06-09 17:57:44,589 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-09 17:57:44,592 | app.py:220 | app_fit: metrics_distributed {'tp': [(1, 357.52979532693354), (2, 364.44397053121367), (3, 336.49396797543324)], 'fp': [(1, 17301.651512407174), (2, 15380.034897246995), (3, 14506.793375740293)], 'tn': [(1, 28276.340518022098), (2, 30184.709965102753), (3, 31077.09

(launch_and_evaluate pid=42754) 1440/1440 - 15s - loss: 0.5859 - tp: 80.0000 - fp: 12758.0000 - tn: 33210.0000 - fn: 32.0000 - accuracy: 0.7224 - precision: 0.0062 - recall: 0.7143 - auc: 0.7745 - prc: 0.0079 - 15s/epoch - 10ms/step [repeated 2x across cluster]
(launch_and_evaluate pid=42754) Hi, the loss is  0.5859057307243347 [repeated 2x across cluster]
(launch_and_evaluate pid=42754)  [repeated 8x across cluster]
(launch_and_evaluate pid=42754) Hi, the loss is  0.5859057307243347
(launch_and_evaluate pid=42754) Hi, the loss is  0.5859057307243347
(launch_and_evaluate pid=42754) Hi, the loss is  0.5859057307243347
(launch_and_evaluate pid=42754) Hi, the loss is  0.5859057307243347


In [77]:
#TODO: This whole cell needs to be cleaned up
# Write out the results of evaluation.

count = 0
for key in (fl_history.metrics_distributed):
    results[count][1] = fl_history.metrics_distributed[key][(num_rounds-1):][0][1]
    count += 1
count = 0
results[9][1] = fl_epochs
results[10][1] = num_rounds
results[11][1] = NUM_CLIENTS * 0.75
results[12][1] = NUM_CLIENTS * 0.25

file = results_path + "history" + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"
results_out = pd.DataFrame(results)
results_out.columns = ['single_server', 'distributed']
results_out.index = ['tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc', 'prc', 'epochs', 'rounds', 'train_clients', 'evaluate_clients']
print(results_out)
pd.DataFrame(results_out).to_csv(file)
print("\nWriting out results.")

                  single_server   distributed
tp                   930.000000    336.493968
fp                 12505.000000  14506.793376
tn                 32214.000000  31077.094538
fn                   431.000000    159.618118
accuracy               0.719271      0.681719
precision              0.069222      0.022622
recall                 0.683321      0.696247
auc                    0.757354      0.732982
prc                    0.084407      0.023046
epochs                10.000000      3.000000
rounds                 1.000000      3.000000
train_clients          1.000000     18.000000
evaluate_clients       1.000000      6.000000

Writing out results.
